In [1]:
import os
import conda

conda_file_dir = conda.__file__
conda_dir = conda_file_dir.split('lib')[0]
proj_lib = os.path.join(os.path.join(conda_dir, 'share'), 'proj')
os.environ["PROJ_LIB"] = proj_lib

from mpl_toolkits.basemap import Basemap

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pygrib
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import PolyCollection
import time

def print_time(t0):
    t1 = time.time()
    return t1-t0

In [2]:
# starting and ending viewazimuths
azim_start = 270
azim_end = 270

# starting and ending view elevations
elev_start = 90
elev_end = 90

# starting and ending zooms
dist_start = 230
dist_end = 230#100

#airport coordinates
lat_DEN = 39.7392
lon_DEN = -104.9903
lat_MIA = 25.7617
lon_MIA = -80.1918

# starting and ending center latitude
lat_start = 38.5
lat_end = 38.5#(lat_DEN+lat_MIA)/2

# starting and ending longitude
lon_start = -97.5
lon_end = -97.5#(lon_DEN+lon_MIA)/2

# starting and ending layer separation
delev_start = 0
delev_end = 0

#latitude windows
degree_window_start = 5
degree_window_end = 5

# delta center degrees
# latitude window for center
# tradoff with dist
ddeg = 1

#save files setup
savepath = '../plots/map/movie/animation/'
filename = 'stage7_'


num_steps = 150

t0 = time.time()

variables = ['Total Cloud Cover']
colors = ['viridis']
ranges = [(0,100)]

for i in range(num_steps):
    delev = delev_start-i*((delev_start-delev_end)/(num_steps-1))
    degree_window = degree_window_start-i*((degree_window_start-degree_window_end)/(num_steps-1))
    hour = str(i-24*(i//24)).zfill(2)
    day = str(1+i//24).zfill(2)
    try:
        grbs = pygrib.open(f'../data/weather/august_full/rap_130_201808{day}_{hour}00_000.grb2')
    except:
        pass
    
    datas = []
    elev = np.arange(len(variables))*delev

    for variable in variables:
        grb = grbs.select(name=variable)[0]
        lats, lons = grb.latlons()
        datas.append(grb['values'])

    map = Basemap(llcrnrlon=-360,llcrnrlat=-90,urcrnrlon=360,urcrnrlat=90,)

    fig = plt.figure(figsize=[10,10],dpi=150)
    ax = Axes3D(fig)

    ax.add_collection3d(map.drawcoastlines(linewidth=0.25))
    ax.add_collection3d(map.drawcountries(linewidth=0.35))

    ax.set_axis_off()
    ax.azim = azim_start-i*((azim_start-azim_end)/(num_steps-1))
    ax.elev = elev_start-i*((elev_start-elev_end)/(num_steps-1))
    ax.dist = dist_start-i*((dist_start-dist_end)/(num_steps-1))
    
    center_lat = lat_start-i*((lat_start-lat_end)/(num_steps-1))
    center_lon = lon_start-i*((lon_start-lon_end)/(num_steps-1))


    for ix,data in enumerate(datas):
        # fourth dimention - colormap
        # create colormap according to x-value (can use any 50x50 array)
        color_dimension = data # change to desired fourth dimension
        minn, maxx = color_dimension.min(), color_dimension.max()
        norm = matplotlib.colors.Normalize(minn, maxx)
        m = plt.cm.ScalarMappable(norm=norm, cmap=colors[ix])
        m.set_array([])
        fcolors = m.to_rgba(color_dimension)
        Z_DEN = np.ones(lats.shape)*elev[ix]+delev/10
        Z_MIA = np.ones(lats.shape)*elev[ix]+delev/10
        mask_DEN = np.ma.masked_where((np.sqrt((lats-lat_DEN)**2 + (lons-lon_DEN)**2) > degree_window), Z_DEN)
        mask_MIA = np.ma.masked_where((np.sqrt((lats-lat_MIA)**2 + (lons-lon_MIA)**2) > degree_window), Z_MIA)
        Z_DEN[np.where(np.ma.getmask(mask_DEN)==True)] = np.nan # or np.where(np.ma.getmask(mask2)==True)] = np.nan
        Z_MIA[np.where(np.ma.getmask(mask_MIA)==True)] = np.nan
        
        ax.plot_surface(lons,lats,Z_DEN,cstride=2,rstride=2,facecolors=fcolors, vmin=ranges[ix][0], vmax=ranges[ix][0], shade=False)
        ax.plot_surface(lons,lats,Z_MIA,cstride=2,rstride=2,facecolors=fcolors, vmin=ranges[ix][0], vmax=ranges[ix][0], shade=False)
        ax.scatter(lon_DEN, lat_DEN, 0,color='r',marker='o',linewidth=10)

    ax.set_xlim3d(center_lon-ddeg,center_lon+ddeg)
    ax.set_ylim3d(center_lat-ddeg,center_lat+ddeg)
    if i < 10:
        fig.savefig(f'{savepath}{filename}0000{i}.png', dpi=fig.dpi)
    elif i >=10 and i < 100:
        fig.savefig(f'{savepath}{filename}000{i}.png', dpi=fig.dpi)
    elif i >=100 and i < 1000:
        fig.savefig(f'{savepath}{filename}00{i}.png', dpi=fig.dpi)
    elif i > 1000:
        fig.savefig(f'{savepath}{filename}0{i}.png', dpi=fig.dpi)
    plt.close()
    print(f'Printed {filename}0000{i}.png in',print_time(t0),'seconds')
    t0 = time.time()

/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Printed stage7_00000.png in 30.190184116363525 seconds
Printed stage7_00001.png in 34.788256883621216 seconds
Printed stage7_00002.png in 46.37772011756897 seconds
Printed stage7_00003.png in 46.64725303649902 seconds
Printed stage7_00004.png in 40.03426671028137 seconds
Printed stage7_00005.png in 158.07436513900757 seconds
Printed stage7_00006.png in 87.53790187835693 seconds
Printed stage7_00007.png in 75.28646922111511 seconds
Printed stage7_00008.png in 157.9108190536499 seconds
Printed stage7_00009.png in 60.85920524597168 seconds
Printed stage7_000010.png in 72.33336663246155 seconds
Printed stage7_000011.png in 35.80965805053711 seconds
Printed stage7_000012.png in 72.45133304595947 seconds
Printed stage7_000013.png in 81.2310209274292 seconds
Printed stage7_000014.png in 73.27893400192261 seconds
Printed stage7_000015.png in 53.99668288230896 seconds
Printed stage7_000016.png in 61.55597186088562 seconds
Printed stage7_000017.png in 56.84909105300903 seconds
Printed stage7_000

In [ ]:
# starting and ending viewazimuths
azim_start = 270
azim_end = 230

# starting and ending view elevations
elev_start = 90
elev_end = 25

# starting and ending zooms
dist_start = 230
dist_end = 130

#airport coordinates
lat_DEN = 39.7392
lon_DEN = -104.9903
lat_MIA = 25.7617
lon_MIA = -80.1918

# starting and ending center latitude
lat_start = 38.5
lat_end = (lat_DEN+lat_MIA)/2

# starting and ending longitude
lon_start = -97.5
lon_end = (lon_DEN+lon_MIA)/2

# starting and ending layer separation
delev_start = 0
delev_end = 0

#latitude windows
degree_window_start = 5
degree_window_end = 5

# delta center degrees
# latitude window for center
# tradoff with dist
ddeg = 1

# flight path height
flight_maxz = 5

# initialize flight list
flight_lat = []
flight_lon = []
flight_z = []

#save files setup
savepath = '../plots/map/movie/animation/'
filename = 'stage8_'


num_steps = 300

t0 = time.time()

variables = ['Total Cloud Cover']
colors = ['viridis']
ranges = [(0,100)]

for i in range(num_steps):
    
    flight_lat.append(lat_DEN-i*((lat_DEN-lat_MIA)/(num_steps-1)))
    flight_lon.append(lon_DEN-i*((lon_DEN-lon_MIA)/(num_steps-1)))
    flight_z.append(((-(1/flight_maxz)*((num_steps-1)/2)**2)**-1)*(i-(num_steps-1)/2)**2+(flight_maxz))
    
    delev = delev_start-i*((delev_start-delev_end)/(num_steps-1))
    degree_window = degree_window_start-i*((degree_window_start-degree_window_end)/(num_steps-1))
    hour = str(i-24*(i//24)).zfill(2)
    day = str(1+i//24).zfill(2)
    try:
        grbs = pygrib.open(f'../data/weather/august_full/rap_130_201808{day}_{hour}00_000.grb2')
    except:
        pass
    
    datas = []
    elev = np.arange(len(variables))*delev

    for variable in variables:
        grb = grbs.select(name=variable)[0]
        lats, lons = grb.latlons()
        datas.append(grb['values'])

    map = Basemap(llcrnrlon=-360,llcrnrlat=-90,urcrnrlon=360,urcrnrlat=90,)

    fig = plt.figure(figsize=[10,10],dpi=150)
    ax = Axes3D(fig)

    ax.add_collection3d(map.drawcoastlines(linewidth=0.25))
    ax.add_collection3d(map.drawcountries(linewidth=0.35))

    ax.set_axis_off()
    ax.azim = azim_start-i*((azim_start-azim_end)/(num_steps-1))
    ax.elev = elev_start-i*((elev_start-elev_end)/(num_steps-1))
    ax.dist = dist_start-i*((dist_start-dist_end)/(num_steps-1))
    
    center_lat = lat_start-i*((lat_start-lat_end)/(num_steps-1))
    center_lon = lon_start-i*((lon_start-lon_end)/(num_steps-1))


    for ix,data in enumerate(datas):
        # fourth dimention - colormap
        # create colormap according to x-value (can use any 50x50 array)
        color_dimension = data # change to desired fourth dimension
        minn, maxx = color_dimension.min(), color_dimension.max()
        norm = matplotlib.colors.Normalize(minn, maxx)
        m = plt.cm.ScalarMappable(norm=norm, cmap=colors[ix])
        m.set_array([])
        fcolors = m.to_rgba(color_dimension)
        Z_DEN = np.ones(lats.shape)*elev[ix]+delev/10
        Z_MIA = np.ones(lats.shape)*elev[ix]+delev/10
        mask_DEN = np.ma.masked_where((np.sqrt((lats-lat_DEN)**2 + (lons-lon_DEN)**2) > degree_window), Z_DEN)
        mask_MIA = np.ma.masked_where((np.sqrt((lats-lat_MIA)**2 + (lons-lon_MIA)**2) > degree_window), Z_MIA)
        Z_DEN[np.where(np.ma.getmask(mask_DEN)==True)] = np.nan # or np.where(np.ma.getmask(mask2)==True)] = np.nan
        Z_MIA[np.where(np.ma.getmask(mask_MIA)==True)] = np.nan
        
        ax.plot_surface(lons,lats,-1*Z_DEN,cstride=2,rstride=2,facecolors=fcolors, vmin=ranges[ix][0], vmax=ranges[ix][0], shade=False, zorder=1)
        ax.plot_surface(lons,lats,-1*Z_MIA,cstride=2,rstride=2,facecolors=fcolors, vmin=ranges[ix][0], vmax=ranges[ix][0], shade=False, zorder=1)
        ax.plot(flight_lon, flight_lat, flight_z, color='r',marker=None,linewidth=3, zorder=2)
        ax.scatter(flight_lon[-1], flight_lat[-1], flight_z[-1],color='r',marker='o',linewidth=10, zorder=3)
        print(flight_z[-1])

    ax.set_xlim3d(center_lon-ddeg,center_lon+ddeg)
    ax.set_ylim3d(center_lat-ddeg,center_lat+ddeg)
    if i < 10:
        fig.savefig(f'{savepath}{filename}0000{i}.png', dpi=fig.dpi)
    elif i >=10 and i < 100:
        fig.savefig(f'{savepath}{filename}000{i}.png', dpi=fig.dpi)
    elif i >=100 and i < 1000:
        fig.savefig(f'{savepath}{filename}00{i}.png', dpi=fig.dpi)
    elif i > 1000:
        fig.savefig(f'{savepath}{filename}0{i}.png', dpi=fig.dpi)
    plt.close()
    print(f'Printed {filename}0000{i}.png in',print_time(t0),'seconds')
    t0 = time.time()

0.0


/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Printed stage8_00000.png in 89.12259197235107 seconds
0.06666592096285306
Printed stage8_00001.png in 188.30926394462585 seconds
0.13288441963736464
Printed stage8_00002.png in 40.362528800964355 seconds
0.19865549602353472
Printed stage8_00003.png in 36.5405797958374 seconds
0.2639791501213633
Printed stage8_00004.png in 80.42832398414612 seconds
0.3288553819308513
Printed stage8_00005.png in 102.5678939819336 seconds
0.3932841914519969
Printed stage8_00006.png in 100.15664577484131 seconds
0.4572655786848019
Printed stage8_00007.png in 92.51560497283936 seconds
0.5207995436292663
Printed stage8_00008.png in 139.15397596359253 seconds
0.5838860862853883
Printed stage8_00009.png in 120.71345615386963 seconds
0.6465252066531697
Printed stage8_000010.png in 133.9938051700592 seconds
0.7087169047326096
Printed stage8_000011.png in 94.53479599952698 seconds
0.770461180523708
Printed stage8_000012.png in 121.98645496368408 seconds
0.831758034026465
Printed stage8_000013.png in 77.9284129142